<a href="https://colab.research.google.com/github/pavlo-da/portfolio-project-2-python/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [3]:
path = '/content/drive/MyDrive/Mate_homework/Portfolio Project 2.csv'
df = pd.read_csv(path)
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Ecuador,desktop,Americas,Organic Search,2,2,session with orders,1
1,2020-11-01,Uruguay,mobile,Americas,Organic Search,2,2,session with orders,1
2,2020-11-01,Kenya,desktop,Africa,Organic Search,2,1,session with orders,1
3,2020-11-01,Serbia,desktop,Europe,Paid Search,2,1,session with orders,1
4,2020-11-01,Malta,mobile,Europe,Social Search,2,2,session with orders,1


In [4]:
print(df['event_name'].unique())

['session with orders' 'new account' 'session' 'user_engagement'
 'session_start' 'first_visit' 'page_view' 'view_promotion' 'view_item'
 'scroll' 'add_shipping_info' 'add_to_cart' 'select_item'
 'view_search_results' 'begin_checkout' 'add_payment_info'
 'select_promotion' 'click' 'view_item_list']


In [5]:
df['event_name'] = df['event_name'].replace('new account', 'new_account')

In [6]:
# List of events we're interested in analyzing
events = [
    'session',
    'add_payment_info',
    'add_shipping_info',
    'begin_checkout',
    'new_account'
]

In [7]:
# Pivot the dataframe to aggregate counts of each event per test and group
pivoted = (
    df[df['event_name'].isin(events)]
    .groupby(['test', 'test_group', 'event_name'])['value']
    .sum()
    .unstack(fill_value=0)
    .reset_index()
)

pivoted

event_name,test,test_group,add_payment_info,add_shipping_info,begin_checkout,new_account,session
0,1,1,1988,3034,3784,3823,45362
1,1,2,2229,3221,4021,3681,45193
2,2,1,2344,3480,4262,4165,50637
3,2,2,2409,3510,4313,4184,50244
4,3,1,3623,5298,9532,5856,70047
5,3,2,3697,5188,9264,5822,70439
6,4,1,3731,5128,12555,8984,105079
7,4,2,3601,4956,12267,8687,105141


In [15]:
# Reorder columns: test identifiers first, then event counts
grouped = pivoted[['test', 'test_group'] + events]
# We'll only test meaningful conversion events
events = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']

In [16]:
# Lists to collect textual and tabular results
text_results = []
table_results = []

# Determine how many distinct tests we have
tests_count = grouped['test'].nunique()

# Loop through each test number
for test_num in range(1, tests_count+1):
    # Extract data for this test
    test_data = grouped[grouped['test'] == test_num]

    # Split into control (group 1) and variant (group 2)
    group_A = test_data[test_data['test_group'] == 1].iloc[0]
    group_B = test_data[test_data['test_group'] == 2].iloc[0]

    # Analyze each event
    for event in events:
        # Numerators: number of users who triggered the event
        conv_A, sess_A = group_A[event], group_A['session']
        # Denominators: total sessions for each group
        conv_B, sess_B = group_B[event], group_B['session']

        # Calculate conversion rates
        rate_A = conv_A / sess_A
        rate_B = conv_B / sess_B

        # Compute % change from Group A to Group B
        metric_change = ((rate_B - rate_A) / rate_A) * 100 if rate_A != 0 else np.nan

        # Perform two-sided z-test for proportions
        count = np.array([conv_A, conv_B])
        nobs = np.array([sess_A, sess_B])
        z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

        # Determine statistical significance at alpha = 0.05
        significant = p_value < 0.05

        # Store results for later display
        table_results.append({
            'Test': test_num,
            'Event': event,
            'Numerator A(Event)': conv_A,
            'Denominator A(Session)': sess_A,
            'Numerator B(Event)': conv_B,
            'Denominator B(Session)': sess_B,
            'Conv. Rate Group 1': rate_A,
            'Conv. Rate Group 2': rate_B,
            'Metric Change %': metric_change,
            'p-value': p_value,
            'z-stat': z_stat,
            'Significant': significant
        })

        # Compute Wilson confidence intervals for each group's rate
        ci_A = proportion_confint(conv_A, sess_A, method='wilson')
        ci_B = proportion_confint(conv_B, sess_B, method='wilson')

        # Compose verdict text based on significance and direction
        if not significant:
            verdict = f"The difference is not statistically significant \n(p-value: {p_value:.4f})"
        else:
            better_group = "Group 1" if rate_A > rate_B else "Group 2"
            verdict = f"Better performance by {better_group} (p-value: {p_value:.4f})"

        # Build human-readable summary
        text_result = f"""
Test {test_num}, Event: {event}
Group 1 conversion rate: {rate_A*100:.1f}% [{ci_A[0]*100:.1f}% - {ci_A[1]*100:.1f}%]
Group 2 conversion rate: {rate_B*100:.1f}% [{ci_B[0]*100:.1f}% - {ci_B[1]*100:.1f}%]
{verdict}
------------------------------------------------"""
        text_results.append(text_result)

for res in text_results:
    print(res)


Test 1, Event: add_payment_info
Group 1 conversion rate: 4.4% [4.2% - 4.6%]
Group 2 conversion rate: 4.9% [4.7% - 5.1%]
Better performance by Group 2 (p-value: 0.0001)
------------------------------------------------

Test 1, Event: add_shipping_info
Group 1 conversion rate: 6.7% [6.5% - 6.9%]
Group 2 conversion rate: 7.1% [6.9% - 7.4%]
Better performance by Group 2 (p-value: 0.0092)
------------------------------------------------

Test 1, Event: begin_checkout
Group 1 conversion rate: 8.3% [8.1% - 8.6%]
Group 2 conversion rate: 8.9% [8.6% - 9.2%]
Better performance by Group 2 (p-value: 0.0029)
------------------------------------------------

Test 1, Event: new_account
Group 1 conversion rate: 8.4% [8.2% - 8.7%]
Group 2 conversion rate: 8.1% [7.9% - 8.4%]
The difference is not statistically significant 
(p-value: 0.1229)
------------------------------------------------

Test 2, Event: add_payment_info
Group 1 conversion rate: 4.6% [4.4% - 4.8%]
Group 2 conversion rate: 4.8% [4.6% - 

In [17]:
results_df = pd.DataFrame(table_results)

# Define column order for clarity
column_order = [
    'Test',
    'Event',
    'Numerator A(Event)',
    'Denominator A(Session)',
    'Numerator B(Event)',
    'Denominator B(Session)',
    'Conv. Rate Group 1',
    'Conv. Rate Group 2',
    'Metric Change %',
    'p-value',
    'z-stat',
    'Significant'
]
results_df = results_df[column_order]

# Format numeric columns for readability
styled_df = results_df.style.format({
    'Conv. Rate Group 1': '{:.4f}',
    'Conv. Rate Group 2': '{:.4f}',
    'Metric Change %': '{:.2f}%',
    'p-value': '{:.4f}',
    'z-stat': '{:.4f}'
})

# Highlight rows where p-value < 0.05 to mark significance
def highlight_significant(val):
    if isinstance(val, (int, float)):
        return 'background-color: green' if val < 0.05 else ''
    return ''

styled_df = styled_df.map(highlight_significant, subset=['p-value'])

styled_df

,Test,Event,Numerator A(Event),Denominator A(Session),Numerator B(Event),Denominator B(Session),Conv. Rate Group 1,Conv. Rate Group 2,Metric Change %,p-value,z-stat,Significant
0,1,add_payment_info,1988,45362,2229,45193,0.0438,0.0493,12.54%,0.0001,-3.9249,True
1,1,add_shipping_info,3034,45362,3221,45193,0.0669,0.0713,6.56%,0.0092,-2.6036,True
2,1,begin_checkout,3784,45362,4021,45193,0.0834,0.0890,6.66%,0.0029,-2.9788,True
3,1,new_account,3823,45362,3681,45193,0.0843,0.0815,-3.35%,0.1229,1.5429,False
4,2,add_payment_info,2344,50637,2409,50244,0.0463,0.0479,3.58%,0.2146,-1.2410,False
5,2,add_shipping_info,3480,50637,3510,50244,0.0687,0.0699,1.65%,0.4780,-0.7096,False
6,2,begin_checkout,4262,50637,4313,50244,0.0842,0.0858,1.99%,0.3406,-0.9529,False
7,2,new_account,4165,50637,4184,50244,0.0823,0.0833,1.24%,0.5560,-0.5888,False
8,3,add_payment_info,3623,70047,3697,70439,0.0517,0.0525,1.47%,0.5201,-0.6432,False
9,3,add_shipping_info,5298,70047,5188,70439,0.0756,0.0737,-2.62%,0.1574,1.4137,False


Based on the results of the A/B test, the following conclusions can be made:

1. For Group 1, the tests showed statistically significant improvements in conversion rates for the events "add_payment_info," "add_shipping_info," and "begin_checkout." All metric changes were statistically significant with low p-values, confirming the significance of the conversion rate differences.

2. For Groups 2 and 3, none of the events showed statistically significant differences in conversion rates, as the p-values were well above the commonly accepted significance level of 0.05.

3. Group 4 showed statistically significant decreases in conversion rates for the events "begin_checkout" and "new_account."

In [20]:
save_path = '/content/drive/My Drive/portfolio_project_2.csv'
results_df.to_csv(save_path, index=False)

[CSV](https://drive.google.com/file/d/1DipeviqPkPBPbmruUWF4Lv1hEVBVJ4aU/view?usp=sharing)

[Tableau](https://public.tableau.com/shared/7HXSFWMSR?:display_count=n&:origin=viz_share_link)